In [1]:
import pandas as pd
import random
import datetime

In [2]:
def random_dt_bw(start_date,end_date):
    days_between = (end_date - start_date).days
    random_num_days = random.randrange(days_between)
    random_dt = start_date + datetime.timedelta(days=random_num_days)
    return random_dt

def generate_data(n=1000):
    items = [f"i_{x}" for x in range(n)]
    start_dates = [random_dt_bw(datetime.date(2020,1,1),datetime.date(2020,9,1)) for x in range(n)]
    end_dates = [x + datetime.timedelta(days=random.randint(1,10)) for x in start_dates]
    
    offerDf = pd.DataFrame({"Item":items,
                            "StartDt":start_dates,
                            "EndDt":end_dates})
    
    transaction_items = [f"i_{random.randint(0,n)}" for x in range(5*n)]
    transaction_dt = [random_dt_bw(datetime.date(2020,1,1),datetime.date(2020,9,1)) for x in range(5*n)]
    sales_amt = [random.randint(0,1000) for x in range(5*n)]
    
    transactionDf = pd.DataFrame({"Item":transaction_items,"TransactionDt":transaction_dt,"Sales":sales_amt})

    return offerDf,transactionDf

In [3]:
offerDf,transactionDf = generate_data(n=100000)

In [4]:
offerDf.head()

,Item,StartDt,EndDt
0,i_0,2020-01-13,2020-01-15
1,i_1,2020-01-03,2020-01-12
2,i_2,2020-05-31,2020-06-05
3,i_3,2020-04-11,2020-04-18
4,i_4,2020-08-20,2020-08-21


In [5]:
transactionDf.head()

,Item,TransactionDt,Sales
0,i_19652,2020-06-25,969
1,i_6048,2020-01-17,150
2,i_28922,2020-08-14,465
3,i_90116,2020-01-10,533
4,i_67713,2020-01-14,227


# Pandas Solution

In [6]:
%%time 

merged_df = pd.merge(offerDf,transactionDf,on='Item')
pandas_solution = merged_df[(merged_df['TransactionDt']>=merged_df['StartDt']) & 
                               (merged_df['TransactionDt']<=merged_df['EndDt'])]

CPU times: user 458 ms, sys: 36 ms, total: 494 ms
Wall time: 494 ms


In [7]:
pandas_solution.head()

,Item,StartDt,EndDt,TransactionDt,Sales
108,i_21,2020-07-31,2020-08-09,2020-08-04,236
123,i_25,2020-02-17,2020-02-20,2020-02-20,935
178,i_37,2020-05-28,2020-06-07,2020-06-02,244
212,i_44,2020-07-27,2020-08-02,2020-07-31,588
256,i_52,2020-08-18,2020-08-25,2020-08-18,495


## PandasSQL Solution

In [8]:
! pip install -U pandasql

Requirement already up-to-date: pandasql in /miniconda3/envs/py36/lib/python3.6/site-packages (0.7.3)
You should consider upgrading via the '/miniconda3/envs/py36/bin/python -m pip install --upgrade pip' command.


In [9]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [10]:
%%time 

q = """
    SELECT A.*,B.TransactionDt,B.Sales
        FROM
            offerDf A
        INNER JOIN
            transactionDf B
        ON 
            A.Item = B.Item AND
            A.StartDt <= B.TransactionDt AND
            A.EndDt >= B.TransactionDt;
         """
pandasSQL_solution = pysqldf(q)


CPU times: user 4.34 s, sys: 1.14 s, total: 5.48 s
Wall time: 5.51 s


In [11]:
pandasSQL_solution.head()

,Item,StartDt,EndDt,TransactionDt,Sales
0,i_21,2020-07-31,2020-08-09,2020-08-04,236
1,i_25,2020-02-17,2020-02-20,2020-02-20,935
2,i_37,2020-05-28,2020-06-07,2020-06-02,244
3,i_44,2020-07-27,2020-08-02,2020-07-31,588
4,i_52,2020-08-18,2020-08-25,2020-08-18,495
